<a href="https://colab.research.google.com/github/maxhormazabal/depencendy_parsing/blob/main/P2_Preprocessing_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

## Instalación de dependencias

In [3]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Conectando con Google Drive para leer el archivo `.py` que contiene las funciones a utilizar.

In [4]:
# Getting access to Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Import our own functions (they are in a .py file on Google Drive)
import os
os.chdir("/content/drive/MyDrive/MASTER")
from nlu_preprocessing_utils import *
from conllu import parse
import tensorflow as tf
import pandas as pd
import numpy as np
import math

## Leyendo fuente de datos desde su origen en github

In [6]:
# English
# 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu'
# https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/en_partut-ud-train.conllu

base_url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/'
file_basename = 'en_partut-ud'
(en_train,en_test,en_val) = readConlluDataset(base_url,file_basename)
en_upo2number, en_number2upo, en_nupos = getUposList(en_train)

Total of different UPOS:  17
{'NOUN': 1, 'ADP': 2, 'DET': 3, 'AUX': 4, 'PART': 5, 'VERB': 6, 'PUNCT': 7, 'PROPN': 8, 'CCONJ': 9, 'ADJ': 10, 'ADV': 11, 'PRON': 12, 'NUM': 13, 'SCONJ': 14, 'X': 15, 'INTJ': 16, 'SYM': 17}


## Transformando la fuente de datos en el dataset inicial

In [7]:
train_df = conlluToDatasetForDependency(en_train,en_upo2number)
test_df = conlluToDatasetForDependency(en_test,en_upo2number)
val_df = conlluToDatasetForDependency(en_val,en_upo2number)

In [8]:
stack_len = 2
buffer_len = 3
(x_train,y_train) = transformByOracle(train_df,stack_len,buffer_len)
(x_test,y_test) = transformByOracle(test_df,stack_len,buffer_len)
(x_val,y_val) = transformByOracle(val_df,stack_len,buffer_len)

In [9]:
# function to save the numpy array as a file and re-use it avoiding preprocessing steps
# It will be save in your current directory (seted in "os.chdir("/content/drive/MyDrive/MASTER")")

# Saving train data
np.save('nlu_data/x_train.npy', x_train) 
np.save('nlu_data/y_train.npy', y_train)

# Saving test data
np.save('nlu_data/x_test.npy', x_test) 
np.save('nlu_data/y_test.npy', y_test)

# Saving val data
np.save('nlu_data/x_val.npy', x_val) 
np.save('nlu_data/y_val.npy', y_val)

In [ ]:
# With the following you can read the file and create the numpy array again
new_x_data = np.load('x_data.npy',allow_pickle=True)
new_action_data = np.load('action_data.npy',allow_pickle=True)